# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [342]:
#imports 
import pandas as pd

In [571]:
#Leitura dos arquivos em Excel

nubank_treinamento = pd.read_excel('tweets_treinamento_p2.xlsx')
nu_treino_sim = nubank_treinamento[nubank_treinamento['Avaliacao'] == 'sim']
nu_treino_nao = nubank_treinamento[nubank_treinamento['Avaliacao'] == 'nao']
nubank_teste = pd.read_excel('tweets_teste_p2.xlsx')

In [572]:
def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t']
    i2 = ['?', '!']
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
        
    tabela[titulo] = a

    return tabela

def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['' , ' ']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova

def tabela_probabilidades(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras'].value_counts(normalize=True)

def tabela_tudo(tabela1, tabela2):
    a = tabela1['palavras']
    b = tabela2['palavras']
    tudo = pd.DataFrame()
    tudo['palavras'] = []
    for lista in a:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    for lista in b:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    
    return tudo

In [573]:
prob_sim = tabela_probabilidades(split_tweet(limpar_caracteres(nu_treino_sim, 'Treinamento'), 'Treinamento'))
prob_nao = tabela_probabilidades(split_tweet(limpar_caracteres(nu_treino_nao, 'Treinamento'), 'Treinamento'))
teste = split_tweet(limpar_caracteres(nubank_teste, 'Teste'), 'Teste')
teste = teste.join(nubank_teste['Avaliacao'])
tudo = tabela_tudo(split_tweet(limpar_caracteres(nu_treino_sim, 'Treinamento'), 'Treinamento'),split_tweet(limpar_caracteres(nu_treino_nao, 'Treinamento'), 'Treinamento'))
total_palavras = len(tudo['palavras'].value_counts())

/Users/Pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [ ]:
def naive_bayes(prob_sim, prob_nao, teste):
for linha in teste:
    p_sim = 1
    p_nao = 1
    for palavra in linha:
        p_sim *= prob_sim[palavra] + 1/total_palavras
        p_nao *= prob_nao[palavra] + 1/total_palavras
    
        
        
p(tweet|sim)*p(sim) - comparar - p(tweet|nao)*p(nao)

(psim[palavra] + 1/total_palavras + ... +)

In [574]:
teste

,palavras,Avaliacao
0,"[queria, saber, pq, digio, e, nubank, investem...",sim
1,"[@samuelbruno47, ainda, nao, temos, previsao, ...",nao
2,"[@jordaanzzzz, opa, fica, tranquilo, !, chama,...",nao
3,"[chegou, meu, @nubank, agora, #sounu, 💜]",sim
4,"[@gaabrielacam1, imagina, !, 💜, que, bom, que,...",nao
5,"[@nubank, pelo, app, ando, mal, sucedido, tamb...",nao
6,"[@nubank, berro, com, o, nome, da, playlist]",nao
7,"[nada, como, ter, que, ligar, 17, dezessete, !...",sim
8,"[nao, to, conseguindo, pagar, minha, fatura, d...",sim
9,"[@lucascaliman, no, app, vc, pode, simular, qu...",nao


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
